In [466]:
#importing libraries
import pandas as pd
import sys 
import urllib
from splinter import Browser
from bs4 import BeautifulSoup

In [467]:
#Launching automated browser & navigating to url of interest
browser=Browser('chrome',headless=False)
url = 'https://www.leaguescup.com/schedule/#club=all&date=2023-07-30'
browser.visit(url)

In [485]:
#getting html script of the matches data and parsing it with beautiful soup
html = browser.html
soup = BeautifulSoup(html,'html.parser')
#storing html with match information found by searching for the class
matchesList=soup.body.find_all(class_='sc-dsHJmm ewkItn sm')

In [576]:
#Scrapping for data needed
#creating empty lists to store data
teams = []
teamAbbs = []
teamGoals = []
homeOrAway=[]
teamPenalties=[]
teamPoints=[]
matchStadiums=[]
matchNumber=[]
opponent=[]
outcome=[]
penaltiesOutcome=[]
matchNum=0
matchID = []
homeTeams=[]
awayTeams=[]
roundCup=[]

#iterating through the html for each match and appending matchinformation into created lists
for match in matchesList:
    matchNum+=1
    try:
        print('Match',matchNum,match.find(class_='sc-iveFHk jsryXz mls-c-status-stamp__status -post').text)
        print(match.find(class_='sc-eDWCr gclhmQ mls-c-match-tile --post').contents[0].find(class_='mls-c-club__shortname').text,match.find(class_='sc-eDWCr gclhmQ mls-c-match-tile --post').contents[2].find(class_='mls-c-club__shortname').text)
       
        homeTeam = match.find(class_='sc-eDWCr gclhmQ mls-c-match-tile --post').contents[0].find(class_='mls-c-club__shortname').text
        homeTeamAbb = match.find(class_='sc-eDWCr gclhmQ mls-c-match-tile --post').contents[0].find(class_='mls-c-club__abbreviation').text
        homeTeamGoal = match.find(class_='sc-eDWCr gclhmQ mls-c-match-tile --post').contents[1].find_all(class_='mls-c-scorebug__score')[0].text
        
        awayTeam = match.find(class_='sc-eDWCr gclhmQ mls-c-match-tile --post').contents[2].find(class_='mls-c-club__shortname').text
        awayTeamAbb = match.find(class_='sc-eDWCr gclhmQ mls-c-match-tile --post').contents[2].find(class_='mls-c-club__abbreviation').text
        awayTeamGoal = match.find(class_='sc-eDWCr gclhmQ mls-c-match-tile --post').contents[1].find_all(class_='mls-c-scorebug__score')[1].text
        
        #id round
        matchStadium = match.find(class_='sc-kgTSHT fROhf').text
        if matchNum <= 45:
            roundCup.append('Qualifying')
        elif matchNum > 45 and matchNum <= 77:
            roundCup.append('Round of 32')
        elif matchNum > 77 and matchNum <= 93:
            roundCup.append('Round of 16')
        elif matchNum > 93 and matchNum <= 97:
            roundCup.append('Quarter Finals')
        elif matchNum > 97 and matchNum <= 99:
            roundCup.append('Semifinals')
        else:
            roundCup.append('Final')
        
        try:
        #penalties results are not easily structured so additional logic is needed
            penalty = (match.find(class_='sc-gScZFl iZdhyg').text.split(':')[0][-2:].strip(),match.find(class_='sc-gScZFl iZdhyg').text.split(':')[1][:2])
            if match.find(class_='sc-gScZFl iZdhyg').text.split(' ')[0]== match.find(class_='sc-eDWCr gclhmQ mls-c-match-tile --post').contents[0].find(class_='mls-c-club__abbreviation').text:
                print('Home Team wins penalties')
                homeTeamPenalty = max(penalty)
                awayTeamPenalty = min(penalty)
                
            else:
                print('Away Team Wins penalties')
                homeTeamPenalty = min(penalty)
                awayTeamPenalty = max(penalty)
                
        except:
            print('No penalties played')
            homeTeamPenalty = 0
            awayTeamPenalty = 0
            
        #calculating team points, teams get addtional point for winning in the penalty kicks    
        homeTeamPoint=0
        awayTeamPoint=0
        
        if homeTeamGoal>awayTeamGoal:
            homeTeamPoint+=3
            homeTeamOutcome='W'
            awayTeamoutcome='L'
            homePOutcome='NA'
            awayPOutcome='NA'
        elif homeTeamGoal<awayTeamGoal:
            awayTeamPoint+=3
            homeTeamOutcome='L'
            awayTeamOutcome='W'
            homePOutcome='NA'
            awayPOutcome='NA'    
        elif homeTeamGoal == awayTeamGoal and homeTeamPenalty>awayTeamPenalty:
            homeTeamPoint+=2
            awayTeamPoint+=1
            homeTeamOutcome='T'
            awayTeamOutcome='T'
            homePOutcome='W'
            awayPOutcome='L'
        elif homeTeamGoal == awayTeamGoal and homeTeamPenalty<awayTeamPenalty:
            homeTeamPoint+=1
            awayTeamPoint+=2
            homeTeamOutcome='T'
            awayTeamOutcome='T'
            homePOutcome='L'
            awayPOutcome='W'
        #general match info
        matchID.append(matchNum)
        homeTeams.append(homeTeam)
        awayTeams.append(awayTeam)
       
        #home match info
        teamPenalties.append(homeTeamPenalty)        
        teamGoals.append(homeTeamGoal)    
        teamAbbs.append(homeTeamAbb)    
        teams.append(homeTeam)    
        teamPoints.append(homeTeamPoint)
        matchStadiums.append(matchStadium)
        homeOrAway.append('Home')
        opponent.append(awayTeam)
        matchNumber.append(matchNum)
        outcome.append(homeTeamOutcome)
        penaltiesOutcome.append(homePOutcome)
        
        #away match info
        teams.append(awayTeam)
        teamAbbs.append(awayTeamAbb)
        teamGoals.append(awayTeamGoal)
        teamPenalties.append(awayTeamPenalty)
        teamPoints.append(awayTeamPoint)
        homeOrAway.append('Away')
        opponent.append(homeTeam)
        matchStadiums.append(matchStadium)
        matchNumber.append(matchNum)
        outcome.append(awayTeamOutcome)
        penaltiesOutcome.append(awayPOutcome)
          
    #if match has not been played then error will be thrown, error is caught and "match has not been played" is outputed     
    except:
        print('Match', matchNum,'has not been played')

Match 1 Final
Cruz Azul Miami
No penalties played
Match 2 Final
Orlando Houston
Home Team wins penalties
Match 3 Final
Austin Mazatlán
No penalties played
Match 4 Final
Dallas Charlotte
Away Team Wins penalties
Match 5 Final
León Vancouver
Home Team wins penalties
Match 6 Final
Montréal Pumas
Home Team wins penalties
Match 7 Final
New York New England
Home Team wins penalties
Match 8 Final
Philadelphia Tijuana
No penalties played
Match 9 Final
Salt Lake Seattle
No penalties played
Match 10 Final
Portland San Jose
No penalties played
Match 11 Final
New York City Atlas
No penalties played
Match 12 Final
Cincinnati Kansas City
Home Team wins penalties
Match 13 Final
Nashville Colorado
No penalties played
Match 14 Final
Puebla Minnesota
No penalties played
Match 15 Final
Columbus St. Louis
No penalties played
Match 16 Final
Miami Atlanta
No penalties played
Match 17 Final
Santos Laguna Houston
Away Team Wins penalties
Match 18 Final
Dallas Necaxa
No penalties played
Match 19 Final
Mazatlán

In [577]:

matchDetailsDic = {'Match_id': matchNumber,
        'Team':teams,
        'Home_Away':homeOrAway,
        'Outcome':outcome,
       'Goals':teamGoals,
        'Opponent':opponent,
       'Team_penalties':teamPenalties,
        'Penalties_outcome':penaltiesOutcome,
       'Team_points': teamPoints,
        'Match_Stadium':matchStadiums,
       }
matchDic = {'Match_ID':matchID,
        'Home_Team':homeTeams,
        'Away_Team':awayTeams,
        'Round':roundCup}
matchDetailsDF = pd.DataFrame(matchDetailsDic)
matches = pd.DataFrame(matchDic)

In [578]:
matches.to_csv('Matches.csv',index=False)

In [579]:
matchDetailsDF.to_csv('MatchDetails.csv',index=False)

In [581]:
league = pd.DataFrame({'League':['Liga MX','MLS','MLS','MLS','MLS','Liga MX','MLS','MLS','Liga MX','MLS','MLS','Liga MX','MLS','MLS','MLS','Liga MX','MLS','MLS','MLS','MLS','MLS','Liga MX','MLS','MLS','MLS','MLS','Liga MX','MLS','MLS','MLS','MLS','Liga MX','Liga MX','Liga MX','MLS','MLS','Liga MX','Liga MX','Liga MX','MLS','Liga MX','MLS','Liga MX','Liga MX','Liga MX','Liga MX','MLS']})
teams = matchDetailsDF[['Team']].drop_duplicates().reset_index(drop=True)
teamsDF = pd.concat([teams,league],axis=1)
teamsDF.to_csv('Teams.csv',index=False)